<a href="https://colab.research.google.com/github/Altaieb-Mohammed/lab_2corse/blob/master/c_lab1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
url = "https://raw.githubusercontent.com/Altaieb-Mohammed/lab_2corse/master/Bank_Customers.csv"
data = pd.read_csv(url)


In [ ]:
print(data.head(5))

print(data.isnull().sum())

print(data.duplicated().sum())


   customer_id  credit_score country  gender  age  tenure    balance  \
0     15634602           619  France  Female   42       2       0.00   
1     15647311           608   Spain  Female   41       1   83807.86   
2     15619304           502  France  Female   42       8  159660.80   
3     15701354           699  France  Female   39       1       0.00   
4     15737888           850   Spain  Female   43       2  125510.82   

   products_number  credit_card  active_member  estimated_salary  churn  
0                1            1              1         101348.88      1  
1                1            0              1         112542.58      0  
2                3            1              0         113931.57      1  
3                2            0              0          93826.63      0  
4                1            1              1          79084.10      0  
customer_id         0
credit_score        0
country             0
gender              0
age                 0
tenure       

In [ ]:

data = data.drop_duplicates()


data = data.fillna(data.median(numeric_only=True))


In [4]:
from sklearn.preprocessing import MinMaxScaler

scaler = MinMaxScaler()
data_scaled = scaler.fit_transform(data[['credit_score', 'age', 'tenure', 'balance', 'estimated_salary']])


In [5]:
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
data_scaled = scaler.fit_transform(data[['credit_score', 'age', 'tenure', 'balance', 'estimated_salary']])


In [2]:
# Remove duplicates
data = data.drop_duplicates()

# (Optional) Handle missing values if any
# data = data.fillna(data.median(numeric_only=True))

# Save to CSV
data.to_csv('bank_customers_cleaned.csv', index=False)


In [3]:
from google.colab import files
files.download('bank_customers_cleaned.csv')


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>